In [1]:
%pwd

'c:\\Users\\91889\\OneDrive\\Desktop\\Diamond-Price-Prediction\\research'

In [2]:
import os

os.chdir("../")

In [3]:
%pwd

'c:\\Users\\91889\\OneDrive\\Desktop\\Diamond-Price-Prediction'

### Config Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

# Data Ingestion
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### Configuration Manager

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    # pull yaml file paths from constants
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        # read yaml files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # create a list of directories
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # create root directory
        create_directories([config.root_dir])

        # create a config for Data Ingestion
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

### Components - Data Ingestion

In [6]:
import os
import zipfile
from pathlib import Path
import urllib.request as request

from mlProject import logger
from mlProject.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        filename: str
        retrieves file using urllib request
        if file already exists, it will log the file size
        Function returns None
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename}  downloaded! with following info: \n{headers}")

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data ingestion directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

### Pipeline

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2024-10-19 01:30:15,745: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-19 01:30:15,753: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-19 01:30:15,753: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-19 01:30:15,753: INFO: common: created directory at: artifacts]
[2024-10-19 01:30:15,761: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-19 01:30:16,440: INFO: 3194312174: artifacts/data_ingestion/data.zip  downloaded! with following info: 
x-amz-id-2: n2xsMblcyANKk+7UHTnZRtNRbVsVbF/hzESfvxXgjACCtgE9DRTLNUwoP1Zp3/IsJr+XZ42yU34=
x-amz-request-id: YMS579T54Y0XMBZS
Date: Fri, 18 Oct 2024 20:00:15 GMT
Last-Modified: Fri, 18 Oct 2024 14:34:38 GMT
ETag: "da6cde3b32d484e3f3fdf6946a108c26"
x-amz-server-side-encryption: AES256
x-amz-version-id: N4DDem7GHy7BJPpDLo4Jnvt2d4woaTlw
Accept-Ranges: bytes
Content-Type: application/zip
Server: AmazonS3
Content-Length: 575577
Connection: close

]
